In [1]:
import anndata
import matplotlib.pyplot as plt
import numpy as np
import scanpy
import scanpy as sc
from scipy.stats import spearmanr
from scvi.data import cortex, smfish
from scvi.external import GIMVI
import scvi
import pandas as pd
from sklearn.model_selection import train_test_split

import random 
random.seed(2023)


[rank: 0] Global seed set to 0


In [2]:
from scipy import stats
import scipy.stats as st
import scipy

In [3]:
def calcualte_pse_correlation(adata_sc, adata_st, celltype, p_value_threshold = 0.05, cor_threshold = 0.5):
    overlap_gene = overlap_gene = list(set(adata_sc.var_names).intersection(adata_st.var_names))
    adata_sc = adata_sc[:,overlap_gene]
    adata_st = adata_st[:,overlap_gene]
    
    cell_type_common = list(set(adata_sc.obs[celltype].unique()).intersection(adata_st.obs[celltype].unique()))
    
    pseudo_st = []
    pseudo_sc = []
    for i in cell_type_common:
        adata1 = adata_st[adata_st.obs[celltype] == i]
        adata2 = adata_sc[adata_sc.obs[celltype] == i]

        pseudo_st.append(np.mean(adata1.X.toarray(), axis = 0))
        pseudo_sc.append(np.mean(adata2.X.toarray(), axis = 0))
    
    pseudo_st = np.array(pseudo_st)
    pseudo_sc = np.array(pseudo_sc)

    cor_pearson = []
    cor_pvalue = []
    for i in range(pseudo_st.shape[1]):
        cor, pval = st.pearsonr(pseudo_st[:,i], pseudo_sc[:,i])
        cor_pearson.append(cor)
        cor_pvalue.append(pval)
        
    information_stat = pd.DataFrame()

    information_stat['pearson'] = cor_pearson
    information_stat['pvalue'] = cor_pvalue
    information_stat.index = adata_st.var_names

    information_stat_update = information_stat.loc[((information_stat['pvalue']<p_value_threshold) & (information_stat['pearson']>cor_threshold))]
    
    return information_stat_update.index

In [4]:
seq_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/tangram/data_smfish/scrnaseq_data.h5ad")
spatial_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/tangram/data_smfish/spatial_data.h5ad")

In [5]:
info_gene = calcualte_pse_correlation(seq_data, spatial_data, 'scClassify')

In [6]:
seq_data = seq_data[:,info_gene]
spatial_data = spatial_data[:,info_gene]

In [7]:
import random 
random.seed(2023)
g1 = list(set(spatial_data .var_names).intersection(seq_data.var_names))
g1  = sorted(g1)
train_g, test_g = train_test_split(g1, test_size=0.33, random_state=2023)
spatial_data_partial = spatial_data[:, train_g].copy()
seq_data = seq_data.copy()

seq_gene_names = seq_data.var_names
n_genes = seq_data.n_vars

# spatial_data_partial has a subset of the genes to train on
spatial_data_partial = spatial_data_partial

# # remove cells with no counts
# scanpy.pp.filter_cells(spatial_data_partial, min_counts=1)
# scanpy.pp.filter_cells(seq_data, min_counts=1)

# setup_anndata for spatial and sequencing data
GIMVI.setup_anndata(spatial_data_partial, labels_key="scClassify", batch_key="batch")
GIMVI.setup_anndata(seq_data, labels_key="scClassify")
# GIMVI.setup_anndata(seq_data, labels_key="graph_cluster_anno")

# spatial_data should use the same cells as our training data
# cells may have been removed by scanpy.pp.filter_cells()
spatial_data = spatial_data[spatial_data_partial.obs_names]

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [8]:
spatial_data

View of AnnData object with n_obs × n_vars = 4530 × 20
    obs: 'x_coord', 'y_coord', 'labels', 'str_labels', 'batch', 'scClassify'

In [9]:

for i in range(0,10):
    scvi.settings.seed = i
    # create our model

    # setup_anndata for spatial and sequencing data
    GIMVI.setup_anndata(spatial_data_partial, labels_key="scClassify", batch_key="batch")
    GIMVI.setup_anndata(seq_data, labels_key="scClassify")
    # GIMVI.setup_anndata(seq_data, labels_key="graph_cluster_anno")

    # spatial_data should use the same cells as our training data
    # cells may have been removed by scanpy.pp.filter_cells()
#     spatial_data = spatial_data[spatial_data_partial.obs_names]
    model = GIMVI(seq_data, spatial_data_partial, n_latent=32)

    # train for 200 epochs
    model.train(500)

    _, fish_imputation_norm = model.get_imputed_values(normalized=True)
    _, fish_imputation_raw = model.get_imputed_values(normalized=False)

    spatial_data_imputed = spatial_data[:, seq_data.var_names]

    spatial_data_imputed.obsm['imputed'] = fish_imputation_norm
    spatial_data_imputed.obsm['imputed_raw'] = fish_imputation_raw
#     del spatial_data_imputed.uns['cell_types']


    spatial_data_imputed.write_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/data_smfish/gimvi_imputed_smfish_filter20010_seed{i}.h5ad")

[rank: 0] Global seed set to 0
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/model/_utils.py:286: UserWarning: This dataset has some empty cells, this might fail inference.Data should be filtered with `scanpy.pp.filter_cells()`
  warnings.warn(
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3.8 /gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/l ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` comma

Epoch 200/200: 100%|██████████| 200/200 [01:31<00:00,  2.19it/s, loss=16.2, v_num=1]

`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 200/200: 100%|██████████| 200/200 [01:31<00:00,  2.18it/s, loss=16.2, v_num=1]

[rank: 0] Global seed set to 1
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/model/_utils.py:286: UserWarning: This dataset has some empty cells, this might fail inference.Data should be filtered with `scanpy.pp.filter_cells()`
  warnings.warn(
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3.8 /gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/l ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` comma

...

Epoch 200/200: 100%|██████████| 200/200 [01:31<00:00,  2.19it/s, loss=16, v_num=1]

[rank: 0] Global seed set to 9
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/model/_utils.py:286: UserWarning: This dataset has some empty cells, this might fail inference.Data should be filtered with `scanpy.pp.filter_cells()`
  warnings.warn(
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3.8 /gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/l ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` comma


Epoch 200/200: 100%|██████████| 200/200 [01:31<00:00,  2.19it/s, loss=16.1, v_num=1]

`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 200/200: 100%|██████████| 200/200 [01:31<00:00,  2.18it/s, loss=16.1, v_num=1]


# Update breast

In [5]:
import anndata
import matplotlib.pyplot as plt
import numpy as np
import scanpy
import scanpy as sc
from scipy.stats import spearmanr
from scvi.data import cortex, smfish
from scvi.external import GIMVI
import scvi
import pandas as pd
from sklearn.model_selection import train_test_split

import random 
random.seed(2023)


In [6]:
from scipy import stats
import scipy.stats as st
import scipy

In [7]:
def calcualte_pse_correlation(adata_sc, adata_st, celltype, p_value_threshold = 0.05, cor_threshold = 0.5):
    overlap_gene = overlap_gene = list(set(adata_sc.var_names).intersection(adata_st.var_names))
    adata_sc = adata_sc[:,overlap_gene]
    adata_st = adata_st[:,overlap_gene]
    
    cell_type_common = list(set(adata_sc.obs[celltype].unique()).intersection(adata_st.obs[celltype].unique()))
    
    pseudo_st = []
    pseudo_sc = []
    for i in cell_type_common:
        adata1 = adata_st[adata_st.obs[celltype] == i]
        adata2 = adata_sc[adata_sc.obs[celltype] == i]

        pseudo_st.append(np.mean(adata1.X.toarray(), axis = 0))
        pseudo_sc.append(np.mean(adata2.X.toarray(), axis = 0))
    
    pseudo_st = np.array(pseudo_st)
    pseudo_sc = np.array(pseudo_sc)

    cor_pearson = []
    cor_pvalue = []
    for i in range(pseudo_st.shape[1]):
        cor, pval = st.pearsonr(pseudo_st[:,i], pseudo_sc[:,i])
        cor_pearson.append(cor)
        cor_pvalue.append(pval)
        
    information_stat = pd.DataFrame()

    information_stat['pearson'] = cor_pearson
    information_stat['pvalue'] = cor_pvalue
    information_stat.index = adata_st.var_names

    information_stat_update = information_stat.loc[((information_stat['pvalue']<p_value_threshold) & (information_stat['pearson']>cor_threshold))]
    
    return information_stat_update.index

In [8]:
# spatial_data =sc.read_h5ad("/gpfs/gibbs/pi/zhao/yl2687/data/spatial_fish_data/xenium_breast/spe_xenium_withMetrics.h5ad")
# seq_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_breast/sce_FFPE_full.h5ad")

In [10]:
for sample_index in range(0,10):
    seq_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_breast/sce_FFPE_full.h5ad")
    spatial_data = sc.read_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/human_breast_simulation/spe_xenium_data_0.1_seed{sample_index}.h5ad")
    seq_data.var_names_make_unique()
    seq_data.obs['scClassify'] = seq_data.obs['graph_cluster_anno'].copy() 

    info_gene = calcualte_pse_correlation(seq_data, spatial_data, 'scClassify')

    seq_data = seq_data[:,info_gene]
    spatial_data = spatial_data[:,info_gene]

    import random 
    random.seed(2023)
    g1 = list(set(spatial_data .var_names).intersection(seq_data.var_names))
    g1  = sorted(g1)
    train_g, test_g = train_test_split(g1, test_size=0.33, random_state=2023)
    spatial_data_partial = spatial_data[:, train_g].copy()
    seq_data = seq_data.copy()

    seq_gene_names = seq_data.var_names
    n_genes = seq_data.n_vars

    # spatial_data_partial has a subset of the genes to train on
    spatial_data_partial = spatial_data_partial

    # # remove cells with no counts
    # scanpy.pp.filter_cells(spatial_data_partial, min_counts=1)
    # scanpy.pp.filter_cells(seq_data, min_counts=1)

    # setup_anndata for spatial and sequencing data
    GIMVI.setup_anndata(spatial_data_partial, labels_key="scClassify")
    GIMVI.setup_anndata(seq_data, labels_key="scClassify")
    # GIMVI.setup_anndata(seq_data, labels_key="graph_cluster_anno")

    # spatial_data should use the same cells as our training data
    # cells may have been removed by scanpy.pp.filter_cells()
    spatial_data = spatial_data[spatial_data_partial.obs_names]


    scvi.settings.seed = 0
    model = GIMVI(seq_data, spatial_data_partial, n_latent = 1024)

    # train for 200 epochs
    model.train(200, batch_size = 1024)

    _, fish_imputation_norm = model.get_imputed_values(normalized=True)
    _, fish_imputation_raw = model.get_imputed_values(normalized=False)

    spatial_data_imputed = spatial_data[:, seq_data.var_names]

    spatial_data_imputed.obsm['imputed'] = fish_imputation_norm
    spatial_data_imputed.obsm['imputed_raw'] = fish_imputation_raw
    #     del spatial_data_imputed.uns['cell_types']


    spatial_data_imputed.write_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/data_breast/gimvi_breast1024_smaple{sample_index}.h5ad")

/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/anndata/_core/anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
[rank: 0] Global seed set to 0
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3.8 /gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/l ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU co

Epoch 200/200: 100%|██████████| 200/200 [01:41<00:00,  2.05it/s, loss=69.5, v_num=1]

`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 200/200: 100%|██████████| 200/200 [01:41<00:00,  1.98it/s, loss=69.5, v_num=1]


/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/anndata/_core/anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/data/_utils.py:119: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  warnings.warn(
[rank: 0] Global seed set to 0
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention

...

Epoch 200/200: 100%|██████████| 200/200 [01:38<00:00,  2.02it/s, loss=71, v_num=1]


/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/anndata/_core/anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/data/_utils.py:119: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  warnings.warn(
[rank: 0] Global seed set to 0
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention

Epoch 200/200: 100%|██████████| 200/200 [01:38<00:00,  2.06it/s, loss=69.1, v_num=1]

`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 200/200: 100%|██████████| 200/200 [01:38<00:00,  2.04it/s, loss=69.1, v_num=1]


# Update mouse brain

In [1]:
import anndata
import matplotlib.pyplot as plt
import numpy as np
import scanpy
import scanpy as sc
from scipy.stats import spearmanr
from scvi.data import cortex, smfish
from scvi.external import GIMVI
import scvi
import pandas as pd
from sklearn.model_selection import train_test_split

import random 
random.seed(2023)


[rank: 0] Global seed set to 0


In [2]:
from scipy import stats
import scipy.stats as st
import scipy

In [3]:
def calcualte_pse_correlation(adata_sc, adata_st, celltype, p_value_threshold = 0.05, cor_threshold = 0.5):
    overlap_gene = overlap_gene = list(set(adata_sc.var_names).intersection(adata_st.var_names))
    adata_sc = adata_sc[:,overlap_gene]
    adata_st = adata_st[:,overlap_gene]
    
    cell_type_common = list(set(adata_sc.obs[celltype].unique()).intersection(adata_st.obs[celltype].unique()))
    
    pseudo_st = []
    pseudo_sc = []
    for i in cell_type_common:
        adata1 = adata_st[adata_st.obs[celltype] == i]
        adata2 = adata_sc[adata_sc.obs[celltype] == i]

        pseudo_st.append(np.mean(adata1.X.toarray(), axis = 0))
        pseudo_sc.append(np.mean(adata2.X.toarray(), axis = 0))
    
    pseudo_st = np.array(pseudo_st)
    pseudo_sc = np.array(pseudo_sc)

    cor_pearson = []
    cor_pvalue = []
    for i in range(pseudo_st.shape[1]):
        cor, pval = st.pearsonr(pseudo_st[:,i], pseudo_sc[:,i])
        cor_pearson.append(cor)
        cor_pvalue.append(pval)
        
    information_stat = pd.DataFrame()

    information_stat['pearson'] = cor_pearson
    information_stat['pvalue'] = cor_pvalue
    information_stat.index = adata_st.var_names

    information_stat_update = information_stat.loc[((information_stat['pvalue']<p_value_threshold) & (information_stat['pearson']>cor_threshold))]
    
    return information_stat_update.index

In [4]:
# spatial_data =sc.read_h5ad("/gpfs/gibbs/pi/zhao/yl2687/data/spatial_fish_data/xenium_breast/spe_xenium_withMetrics.h5ad")
# seq_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_breast/sce_FFPE_full.h5ad")

In [5]:
import gc
for sample_index in range(0,10):
    seq_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_brain/aibs_mouse_ctx-hpf_smartseq_sce.h5ad")
    spatial_data = sc.read_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/data_brain/spe_xenium_data_0.1_seed{sample_index}.h5ad")
    seq_data.var_names_make_unique()
    seq_data.obs['scClassify'] = seq_data.obs['cell_type_alias_label2'].copy() 

    info_gene = calcualte_pse_correlation(seq_data, spatial_data, 'scClassify')

    seq_data = seq_data[:,info_gene]
    spatial_data = spatial_data[:,info_gene]

    import random 
    random.seed(2023)
    g1 = list(set(spatial_data.var_names).intersection(seq_data.var_names))
    g1  = sorted(g1)
    train_g, test_g = train_test_split(g1, test_size=0.33, random_state=2023)
    spatial_data_partial = spatial_data[:, train_g].copy()
    seq_data = seq_data.copy()

    seq_gene_names = seq_data.var_names
    n_genes = seq_data.n_vars

    # spatial_data_partial has a subset of the genes to train on
    spatial_data_partial = spatial_data_partial

    # # remove cells with no counts
    # scanpy.pp.filter_cells(spatial_data_partial, min_counts=1)
    # scanpy.pp.filter_cells(seq_data, min_counts=1)

    # setup_anndata for spatial and sequencing data
    GIMVI.setup_anndata(spatial_data_partial, labels_key="scClassify")
    GIMVI.setup_anndata(seq_data, labels_key="scClassify")
    # GIMVI.setup_anndata(seq_data, labels_key="graph_cluster_anno")

    # spatial_data should use the same cells as our training data
    # cells may have been removed by scanpy.pp.filter_cells()
    spatial_data = spatial_data[spatial_data_partial.obs_names]


    scvi.settings.seed = 0
    model = GIMVI(seq_data, spatial_data_partial, n_latent = 1024)

    # train for 200 epochs
    model.train(300, batch_size = 4096)

    _, fish_imputation_norm = model.get_imputed_values(normalized=True)
    _, fish_imputation_raw = model.get_imputed_values(normalized=False)

    spatial_data_imputed = spatial_data[:, seq_data.var_names]

    spatial_data_imputed.obsm['imputed'] = fish_imputation_norm
    spatial_data_imputed.obsm['imputed_raw'] = fish_imputation_raw
    #     del spatial_data_imputed.uns['cell_types']

    gc.collect()
    spatial_data_imputed.write_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/data_brain/gimvi_mousebrain1024300_smaple{sample_index}.h5ad")

/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/data/_utils.py:119: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  warnings.warn(
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/data/_utils.py:172: UserWarning: Category 35 in adata.obs['_scvi_labels'] has fewer than 3 cells. Models may not train properly.
  warnings.warn(
[rank: 0] Global seed set to 0
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `sr

Epoch 300/300: 100%|██████████| 300/300 [02:05<00:00,  2.39it/s, loss=127, v_num=1] 

`Trainer.fit` stopped: `max_epochs=300` reached.


Epoch 300/300: 100%|██████████| 300/300 [02:05<00:00,  2.38it/s, loss=127, v_num=1]


/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/data/_utils.py:119: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  warnings.warn(
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/data/_utils.py:172: UserWarning: Category 9 in adata.obs['_scvi_labels'] has fewer than 3 cells. Models may not train properly.
  warnings.warn(
[rank: 0] Global seed set to 0
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lig

...

Epoch 300/300: 100%|██████████| 300/300 [02:09<00:00,  2.32it/s, loss=138, v_num=1]


/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/data/_utils.py:119: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  warnings.warn(
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/data/_utils.py:172: UserWarning: Category 22 in adata.obs['_scvi_labels'] has fewer than 3 cells. Models may not train properly.
  warnings.warn(
[rank: 0] Global seed set to 0
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Li

Epoch 300/300: 100%|██████████| 300/300 [02:10<00:00,  2.28it/s, loss=135, v_num=1] 

`Trainer.fit` stopped: `max_epochs=300` reached.


Epoch 300/300: 100%|██████████| 300/300 [02:10<00:00,  2.30it/s, loss=135, v_num=1]
